## House Plant Analysis 

This notebook will cover the initial exploratory data analysis (step 1) of the houseplant database generated. Following this, feature engineering (step 2) will be performed for two goals. First is to perform dimensionality reduction (step 3) as a way to visualise similar plants (ultimate goal to make some 2D scatter like graphs doing this). Second goal is to build a content based recommender engine with this dataset.

#### Step 0: Setup

In [2]:
import pandas as pd
import sqlite3

DATABASE_LOC = r"C:\Users\Rory Crean\Dropbox (lkgroup)\Backup_HardDrive\Postdoc\PyForFun\House_Plant_Recommender\Database\house_plants.db"

In [4]:
conn = sqlite3.connect(DATABASE_LOC)
c = conn.cursor()
df = pd.read_sql_query("SELECT * FROM plant_raw_data", conn)
c.close()
df.head()

,Plant_Name,Common_Names,Plant_Type,Family,Zones,Native_Range,Heights,Spreads,Bloom_Times,Bloom_Description,sunlight,Watering,Maintenance,Flowers,Leafs,Fruits
0,Aechmea,"urn plant,silver vase plant",Epiphyte,Bromeliaceae,10 to 11,Brazil,1.00 to 3.00 feet,1.00 to 2.00 feet,Seasonal bloomer,Violet to red with pink bracts,Part shade,Medium,Medium,Showy,Evergreen,None
1,Ardisia crenata,"hen's eyes,coralberry,spiceberry,scratchthroat...",Broadleaf evergreen,Primulaceae,8 to 10,Japan to Northern India,4.00 to 5.00 feet,4.00 to 5.00 feet,May to June,Pinkish-white,Part shade to full shade,Medium,Medium,Showy,Evergreen,Showy
2,Euphorbia milii,"christplant,Christ plant,Christ thorn,crown of...",Broadleaf evergreen,Euphorbiaceae,9 to 11,Madagascar,3.00 to 6.00 feet,1.50 to 3.00 feet,Seasonal bloomer,Green subtended by red or yellow bracts,Full sun,Dry to medium,Medium,Showy,Evergreen,None
3,Ficus elastica,"Indian rubberplant,India rubber plant,rubber p...",Broadleaf evergreen,Moraceae,10 to 12,Southeastern Asia,50.00 to 100.00 feet,50.00 to 100.00 feet,Rarely flowers indoors,None,Part shade,Medium,Low,Insignificant,Evergreen,None
4,Woodsia obtusa,"blunt-lobed woodsia,common woodsia,large woodsia",Fern,Woodsiaceae,4 to 8,North America,1.00 to 1.50 feet,2.00 to 2.50 feet,Non-flowering,Non-flowering,Part shade to full shade,Medium,Medium,None,None,None


#### Part 1: Data exploration


In [ ]:
# What can be dropped straight away?
# ["Common_Names", ]

#### Part 2: Feature Engineering 
Based on the insights above, in this section the dataframe will be modified accordingly to build the features required for each plant.

#### Part 3.1: Modelling - Dimensionality reduction
This section will focus on using dimensionality reduction as a way to visualise the relationship between each plant.

#### Part 3.2: Modelling - Recommender engine

Building a content based recommender enginer of course as no user data available 